<a href="https://colab.research.google.com/github/suryautharakumar/DAS7000_Data_Analytics_and_Visualisation/blob/main/Stu_Week_7_Workshop_student_activity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Case Study: MarketPulse Electronics - Interactive Data-Driven Sales Insights

MarketPulse Electronics - a rapidly expanding mid-sized electronics retailer - operates across New York, San Francisco, Boston, Los Angeles, and Seattle.

The company has completed a full year of digital sales tracking across multiple product categories - including phones, accessories, computers, and consumer electronics.

Now, the management wants to explore sales patterns interactively to answer complex questions about seasonality, pricing, customer behavior, and regional trends.

As a Data Analyst Consultant, you have been tasked to design an interactive analytics using Plotly Express and Plotly Graph Objects, that enables management to make data-driven decisions.

Dataset Overview

Dataset Used: Sales-Analysis-Dataset

Files: 12 monthly CSV files (January to December)

You need to answer for following business questions (Interactive Focus)

1. Seasonal Revenue Behavior

How do total monthly revenues fluctuate throughout the year?
Can we identify key peaks for example- holiday months and dips interactively?

Hint:

Derive a Month column from Order Date.

Create a dynamic line plot with tooltips and a range slider (go.Scatter).

Highlight months with sudden revenue surges.

2.  City-Wise Sales Distribution

Which U.S. cities bring in the highest total revenue, and how does the sales share differ by region?

Hint:

Extract City from Purchase Address.

Use a Plotly pie chart or choropleth map for interactive exploration.

Hover over each city to view total revenue and average order value.

3. Product Category Comparison

Which products or product types perform best across the year?
Can we compare unit sales and total revenue side-by-side?

Hint:

Use a double bar chart that compare Quantity Ordered vs Total Sales.

Allow users to hover to see revenue-per-item.

4. Hourly Purchase Patterns

During what times of day do customers make the most purchases?
Are late-night or morning sales more common?

Hint:

Extract the Hour from Order Date.

Use an interactive area or histogram plot (px.area or px.histogram).

Enable zooming to see micro trends (e.g., lunchtime spikes).

5. Revenue Distribution per Order

Visualize how transaction revenues vary across the dataset. Are most orders small or do we have a few large, high-value orders?

Hint:

Create a bubble scatter plot (px.scatter) with:

X-axis → Price Each

Y-axis → Quantity Ordered

Bubble size → Total Revenue

Hover to show product names and revenue.


Hints for Students

Combine 12 CSV files using:

all_data = pd.concat([pd.read_csv(f"Sales_{month}.csv") for month in months])


Convert Order Date to datetime:

all_data['Order Date'] = pd.to_datetime(all_data['Order Date'], errors='coerce')


Derive helper columns:

all_data['Month'] = all_data['Order Date'].dt.month
all_data['Hour'] = all_data['Order Date'].dt.hour
all_data['Sales'] = all_data['Quantity Ordered'] * all_data['Price Each']


Extract city names:

all_data['City'] = all_data['Purchase Address'].apply(lambda x: x.split(',')[1])


Use Plotly Express for quick interactivity:

import plotly.express as px


Add range sliders and hover info for richer visualised diagrams.

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#plt.style.use('seaborn-darkgrid')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
%cd '/content/drive/My Drive/Colab/'

/content/drive/My Drive/Colab


In [15]:
%ls

'CMP7005_Programming_for_Data Analysis'/
 Coimbatore_data.csv
 Consumer_Airfare_Report__Table_2_-_Top_1_000_City-Pair_Markets.csv
 dataset_02052023.xlsx
 insurance.csv
 insurance_preprocessed.csv
 sales_report/
 Stu_MiniMart_Sales_Report_Problem.ipynb
 temp.py
 Week_2_workshop_demo.ipynb
 World_GDP.csv


In [14]:
import os

path = 'sales_report'   # Folder where the 12 monthly files are stored
files = [file for file in os.listdir(path) if file.endswith('.csv')]

all_data = pd.DataFrame()
for file in files:
    df = pd.read_csv(os.path.join(path, file))
    all_data = pd.concat([all_data, df])

all_data.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,236670,Wired Headphones,2,11.99,08/31/19 22:21,"359 Spruce St, Seattle, WA 98101"
1,236671,Bose SoundSport Headphones,1,99.99,08/15/19 15:11,"492 Ridge St, Dallas, TX 75001"
2,236672,iPhone,1,700.0,08/06/19 14:40,"149 7th St, Portland, OR 97035"
3,236673,AA Batteries (4-pack),2,3.84,08/29/19 20:59,"631 2nd St, Los Angeles, CA 90001"
4,236674,AA Batteries (4-pack),2,3.84,08/15/19 19:53,"736 14th St, New York City, NY 10001"


In [16]:
all_data.shape

(186850, 6)

In [17]:
all_data.dropna(inplace=True)
all_data = all_data[~all_data['Order Date'].str.contains('Order Date')]

In [18]:
all_data['Order Date'] = pd.to_datetime(all_data['Order Date'])
all_data['Month'] = all_data['Order Date'].dt.month
all_data['Sales'] = all_data['Quantity Ordered'].astype(float) * all_data['Price Each'].astype(float)

/tmp/ipython-input-4126898306.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  all_data['Order Date'] = pd.to_datetime(all_data['Order Date'])


In [92]:
import plotly.graph_objects as go

monthly_sales = all_data.groupby('Month')['Sales'].sum().reset_index()

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=monthly_sales['Month'],
    y=monthly_sales['Sales'],
    mode='lines+markers',
    line=dict(color='#0CC97E', width=2),
    marker=dict(size=10, symbol='16'),
    name='Monthly Revenue',
    hovertemplate='<b>Month %{x}</b><br>Revenue: $%{y:,.0f}<extra></extra>'
))

top_months = monthly_sales.sort_values('Sales', ascending=False).head(2)

fig.add_trace(go.Scatter(
    x=top_months['Month'],
    y=top_months['Sales'],
    mode='markers+text',
    text=['Peak Month'] * len(top_months),
    textposition='top center',
    marker=dict(color='#860CC9', size=10, symbol='star'),
    name='Revenue Surge'
))

fig.update_layout(
    title='Seasonal Revenue Trends by Month',
    xaxis_title='Month',
    yaxis_title='Total Revenue ($)',
    xaxis=dict(
        tickmode='array',
        tickvals=list(range(1, 13)),
        ticktext=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                  'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
        rangeslider=dict(visible=True),
    ),
    template='plotly_white',
    hovermode='x unified'
)

fig.show()
